<a href="https://colab.research.google.com/github/ykitaguchi77/YouTube_DL/blob/main/Faster_Whisper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U yt-dlp
!pip install -U faster_whisper

In [ ]:
import subprocess
from faster_whisper import WhisperModel

YOUTUBE_ID = "TOVAc1YCgag" # Youtube ID
AUDIO_FILE_NAME = f"{YOUTUBE_ID}.mp3"

# Download audio from Youtube
def dl_yt(yt_url):
    subprocess.run(f"yt-dlp -x --audio-format mp3 -o {AUDIO_FILE_NAME} {yt_url}", shell=True)

dl_yt(f"https://youtu.be/{YOUTUBE_ID}")


model_size = "large-v2"

# GPU, FP16で実行
model = WhisperModel(model_size, device="cuda", compute_type="float16")
# GPU, INT8で実行
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# CPU, FP16で実行
# model = WhisperModel(model_size, device="cpu", compute_type="int8")

segments, info = model.transcribe(AUDIO_FILE_NAME, beam_size=5)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))


In [ ]:
segments, info = model.transcribe(AUDIO_FILE_NAME, beam_size=5)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

for segment in segments:
    print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

In [ ]:
import subprocess
from faster_whisper import WhisperModel

YOUTUBE_ID = "TOVAc1YCgag"  # Youtube ID
AUDIO_FILE_NAME = f"{YOUTUBE_ID}.mp3"

# Download audio from Youtube
def dl_yt(yt_url):
    subprocess.run(f"yt-dlp -x --audio-format mp3 -o {AUDIO_FILE_NAME} {yt_url}", shell=True)

dl_yt(f"https://youtu.be/{YOUTUBE_ID}")

model_size = "large-v2"

# GPU, FP16で実行
model = WhisperModel(model_size, device="cuda", compute_type="float16")
# GPU, INT8で実行
# model = WhisperModel(model_size, device="cuda", compute_type="int8_float16")
# CPU, FP16で実行
# model = WhisperModel(model_size, device="cpu", compute_type="int8")

segments, info = model.transcribe(AUDIO_FILE_NAME, beam_size=5)

print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

# Open a text file for writing
with open(f"{YOUTUBE_ID}_transcription.txt", "w", encoding="utf-8") as file:
    for segment in segments:
        # Write only the text of each segment to the file
        file.write(f"{segment.text}\n")
        print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

print(f"Transcription saved to {YOUTUBE_ID}_transcription.txt")


In [13]:
import subprocess
import re
from faster_whisper import WhisperModel

YOUTUBE_ID = "TOVAc1YCgag"  # Youtube ID
AUDIO_FILE_NAME = f"{YOUTUBE_ID}.mp3"

# Download audio and metadata from Youtube
def dl_yt(yt_url):
    # Download audio
    subprocess.run(f"yt-dlp -x --audio-format mp3 -o {AUDIO_FILE_NAME} {yt_url}", shell=True)
    # Get metadata
    result = subprocess.run(f"yt-dlp -j {yt_url}", shell=True, capture_output=True, text=True)
    return result.stdout

metadata = dl_yt(f"https://youtu.be/{YOUTUBE_ID}")

# Extract title from metadata
import json
metadata_json = json.loads(metadata)
title = metadata_json.get("title", YOUTUBE_ID)

# Sanitize the title to be used as a filename
safe_title = re.sub(r'[\\/*?:"<>|]', "", title)

TRANSCRIPTION_FILE_NAME = f"{safe_title}_transcription.txt"

model_size = "large-v2"
model = WhisperModel(model_size, device="cuda", compute_type="float16")
segments, info = model.transcribe(AUDIO_FILE_NAME, beam_size=5)

# Print detected language
print("Detected language '%s' with probability %f" % (info.language, info.language_probability))

# Write the transcription to a file
with open(TRANSCRIPTION_FILE_NAME, "w", encoding="utf-8") as file:
    for segment in segments:
        file.write(f"{segment.text}\n")
        print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))

print(f"Transcription saved to {TRANSCRIPTION_FILE_NAME}")


Detected language 'ja' with probability 0.997559
[0.00s -> 4.80s] モテない男は知らない女の子が好きになるきっかけ3選について話していきたいとおもいます
[4.80s -> 5.40s] どうもヨシです
[5.40s -> 7.60s] 皆さんですねこんなことになってませんかと
[7.60s -> 10.70s] 初対面で強い男をアピってもうまくいかないですと
[10.70s -> 13.00s] おらついたら怖いと言われましたとか
[13.00s -> 14.70s] 皆さんねこういう経験あると思います
[14.70s -> 17.10s] 致命的すぎる間違いをしているので
[17.10s -> 19.40s] 今回はそれを解説していきたいと思います
[19.40s -> 20.50s] 動画を見るメリットですね
[20.50s -> 26.10s] 女の子が好きになる過程と人の感情の動かし方の基礎が分かるようになっております
[26.10s -> 29.20s] なんで人の感情を動かさなあかんのって話もしてるんで
[29.20s -> 31.90s] ぜひですね今回の内容を覚えていってください
[31.90s -> 35.60s] 恋愛をなめくさってる男がまだ世の中には多いです
[35.60s -> 37.90s] そういうねバカどもを皆さん立ち抜いて
[37.90s -> 40.40s] さっさといい女の子とイチャイチャしましょう
[40.40s -> 42.70s] 今回の動画の内容を知らないデメリットですね
[42.70s -> 47.00s] 相手に応じての需要がわからずあっさり情報を拾っては捨てる
[47.00s -> 49.90s] 何も自分の能力がつかない60歳定年と
[49.90s -> 51.70s] はい言いすぎですけども
[51.70s -> 53.40s] 具体的なフレーズとかっていうのは
[53.40s -> 55.90s] 意味ないぜって言ってるのはこれのせいです
[55.90s -> 57.70s] 拾ってきては使って
[57.70s -> 59.60s] あー意味ねーやって捨てて
[59.60s -> 62.80s] 永遠ってやってると本当に何の実力もつかないで
[6